# Introduction

This notebook contains the same content as "criteo_keras.py" but in a notebook(interactive) form.

The dataset used here is from Criteo clicklog dataset. It's preprocessed by DLRM(https://github.com/NVIDIA/DeepLearningExamples/tree/master/TensorFlow2/Recommendation/DLRM/preproc) ETL job on Spark.

We provide a small size sample data in `sample_data` folder

The data schema after the DLRM ETL: also 40 columns -- 1 label column, 39 numerical feature columns. (Categorical columns are processed)

User can either take the same routine to get the data or use your own preprocessed data and change the DL model accordingly.

`Please note: The following demo is dedicated for DGX-2 machine(with V100 GPUs).` We optimized the whole workflow on DGX-2 and it's not guaranteed that it can run successfully on other type of machines.

### import necessary libraries

In [1]:
import argparse
import math
import pprint
import sys
# This needs to happen first to avoid pyarrow serialization errors.
from pyspark.sql import SparkSession

# Make sure pyarrow is referenced before anything else to avoid segfault due to conflict
# with TensorFlow libraries.  Use `pa` package reference to ensure it's loaded before
# functions like `deserialize_model` which are implemented at the top level.
# See https://jira.apache.org/jira/browse/ARROW-3346
import pyarrow as pa

import horovod
import horovod.tensorflow.keras as hvd
import tensorflow as tf
from horovod.spark.common.backend import SparkBackend
from tensorflow.keras.layers import BatchNormalization, Input, Embedding, Concatenate, Dense, Flatten
from tensorflow.keras.layers.experimental.preprocessing import CategoryEncoding

### set some macros

xxx_DATALOADER is the switch to control which dataloader we will use,
xxx_COLUMNS are only used in NVTabular dataloader.

In [1]:
PETASTORM_DATALOADER = 'petastorm'
NVTABULAR_DATALOADER = 'nvtabular'

CONTINUOUS_COLUMNS = [f'i{i}' for i in range(13)]
CATEGORICAL_COLUMNS = [f'c{c}' for c in range(26)]
ALL_COLUMNS = CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS
LABEL_COLUMNS = ['clicked']

The "dimensions" contains the count of distinct value for those categorical column 'after' DLRM ETL.

This will be used to build embedding layers for our model.

In [3]:
def get_category_dimensions(spark, data_dir):
    df = spark.read.csv(f'{data_dir}/dimensions/*.csv', header=True).toPandas()
    dimensions = df.to_dict('records')[0]
    pprint.pprint(dimensions)
    return dimensions

### Build the model

The model is composed by mainly 2 parts: embedding layers and fully-connected layers.

In [4]:
def build_model(dimensions, args):
    
    inputs = {
        **{i: Input(shape=(1,), name=i, dtype=tf.float32) for i in CONTINUOUS_COLUMNS},
        **{c: Input(shape=(1,), name=c, dtype=tf.int32) for c in CATEGORICAL_COLUMNS}
    }

    one_hots = []
    embeddings = []
    for c in CATEGORICAL_COLUMNS:
        dimension = int(dimensions[c]) + 1
        # dimension <= 128, smaller size for demo
        if dimension <= 8:
            one_hots.append(CategoryEncoding(num_tokens=dimension, name=f'one_hot_{c}')(inputs[c]))
        else:
            # embedding_size = int(math.floor(0.6 * dimension ** 0.25)), smaller model size for demo
            embedding_size = 8
            embeddings.append(Embedding(input_dim=dimension,
                                        output_dim=embedding_size,
                                        input_length=1,
                                        name=f'embedding_{c}')(inputs[c]))

    x = Concatenate(name='embeddings_concat')(embeddings)
    x = Flatten(name='embeddings_flatten')(x)
    x = Concatenate(name='inputs_concat')([x] + one_hots + [inputs[i] for i in CONTINUOUS_COLUMNS])
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(32, activation='relu')(x)
    output = Dense(1, activation='sigmoid', name='output')(x)
    model = tf.keras.Model(inputs=[inputs[c] for c in ALL_COLUMNS], outputs=output)
    if hvd.rank() == 0:
        model.summary()

    opt = tf.keras.optimizers.Adam(learning_rate=args.learning_rate)
    opt = hvd.DistributedOptimizer(opt)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

    return model

### Set train function

The train_fn is the function that will execute across all Horovod workers(also say Spark executors in our case).

We use `CUDA_VISIBLE_DEVICES` to control the GPU resources to avoid overlapping.

In [5]:
def train_fn(dimensions, train_rows, val_rows, args):
    # Make sure pyarrow is referenced before anything else to avoid segfault due to conflict
    # with TensorFlow libraries.  Use `pa` package reference to ensure it's loaded before
    # functions like `deserialize_model` which are implemented at the top level.
    # See https://jira.apache.org/jira/browse/ARROW-3346
    pa

    import atexit
    import horovod.tensorflow.keras as hvd
    from horovod.spark.task import get_available_devices
    import os
    import tempfile
    import tensorflow as tf
    import tensorflow.keras.backend as K
    import shutil

    gpus = get_available_devices()
    if gpus:
        os.environ['CUDA_VISIBLE_DEVICES'] = gpus[0]
    if args.dataloader == NVTABULAR_DATALOADER:
        os.environ['TF_MEMORY_ALLOCATION'] = '0.85'
        from nvtabular.loader.tensorflow import KerasSequenceLoader

    # Horovod: initialize Horovod inside the trainer.
    hvd.init()

    # Horovod: restore from checkpoint, use hvd.load_model under the hood.
    model = build_model(dimensions, args)

    # Horovod: adjust learning rate based on number of processes.
    scaled_lr = K.get_value(model.optimizer.lr) * hvd.size()
    K.set_value(model.optimizer.lr, scaled_lr)

    # Horovod: print summary logs on the first worker.
    verbose = 1 if hvd.rank() == 0 else 0

    callbacks = [
        # Horovod: broadcast initial variable states from rank 0 to all other processes.
        # This is necessary to ensure consistent initialization of all workers when
        # training is started with random weights or restored from a checkpoint.
        hvd.callbacks.BroadcastGlobalVariablesCallback(root_rank=0),

        # Horovod: average metrics among workers at the end of every epoch.
        #
        # Note: This callback must be in the list before the ReduceLROnPlateau,
        # TensorBoard, or other metrics-based callbacks.
        hvd.callbacks.MetricAverageCallback(),

        # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
        # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
        # the first five epochs. See https://arxiv.org/abs/1706.02677 for details.
        hvd.callbacks.LearningRateWarmupCallback(initial_lr=scaled_lr, warmup_epochs=5, verbose=verbose),

        # Reduce LR if the metric is not improved for 10 epochs, and stop training
        # if it has not improved for 20 epochs.
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_auc', patience=10, verbose=verbose),
        tf.keras.callbacks.EarlyStopping(monitor='val_auc', mode='min', patience=20, verbose=verbose),
        tf.keras.callbacks.TerminateOnNaN(),

        # Log Tensorboard events.
        tf.keras.callbacks.TensorBoard(log_dir=args.logs_dir, write_steps_per_second=True, update_freq=10)
    ]

    # Horovod: save checkpoints only on the first worker to prevent other workers from corrupting them.
    if hvd.rank() == 0:
        ckpt_dir = tempfile.mkdtemp()
        ckpt_file = os.path.join(ckpt_dir, 'checkpoint.h5')
        atexit.register(lambda: shutil.rmtree(ckpt_dir))
        callbacks.append(tf.keras.callbacks.ModelCheckpoint(
            ckpt_file, monitor='val_auc', mode='min', save_best_only=True))

    if args.dataloader == PETASTORM_DATALOADER:
        from petastorm import make_batch_reader
        from petastorm.tf_utils import make_petastorm_dataset

        # Make Petastorm readers.
        with make_batch_reader(f'{args.data_dir}/train',
                               num_epochs=None,
                               cur_shard=hvd.rank(),
                               shard_count=hvd.size(),
                               hdfs_driver='libhdfs') as train_reader:
            with make_batch_reader(f'{args.data_dir}/val',
                                   num_epochs=None,
                                   cur_shard=hvd.rank(),
                                   shard_count=hvd.size(),
                                   hdfs_driver='libhdfs') as val_reader:
                # Convert readers to tf.data.Dataset.
                train_ds = make_petastorm_dataset(train_reader) \
                    .unbatch() \
                    .shuffle(10 * args.batch_size) \
                    .batch(args.batch_size) \
                    .map(lambda x: (tuple(getattr(x, c) for c in ALL_COLUMNS), x.clicked))

                val_ds = make_petastorm_dataset(val_reader) \
                    .unbatch() \
                    .batch(args.batch_size) \
                    .map(lambda x: (tuple(getattr(x, c) for c in ALL_COLUMNS), x.clicked))

                history = model.fit(train_ds,
                                    validation_data=val_ds,
                                    steps_per_epoch=int(train_rows / args.batch_size / hvd.size()),
                                    validation_steps=int(val_rows / args.batch_size / hvd.size()),
                                    callbacks=callbacks,
                                    verbose=verbose,
                                    epochs=args.epochs)

    else:
        import cupy

        def seed_fn():
            """
            Generate consistent dataloader shuffle seeds across workers
            Reseeds each worker's dataloader each epoch to get fresh a shuffle
            that's consistent across workers.
            """
            min_int, max_int = tf.int32.limits
            max_rand = max_int // hvd.size()
            # Generate a seed fragment on each worker
            seed_fragment = cupy.random.randint(0, max_rand).get()
            # Aggregate seed fragments from all Horovod workers
            seed_tensor = tf.constant(seed_fragment)
            reduced_seed = hvd.allreduce(seed_tensor, name="shuffle_seed", op=hvd.Sum)
            return reduced_seed % max_rand

        train_ds = KerasSequenceLoader(
            f'{args.data_dir}/train',
            batch_size=args.batch_size,
            label_names=LABEL_COLUMNS,
            cat_names=CATEGORICAL_COLUMNS,
            cont_names=CONTINUOUS_COLUMNS,
            engine="parquet",
            shuffle=True,
            buffer_size=0.06,  # how many batches to load at once
            parts_per_chunk=1,
            global_size=hvd.size(),
            global_rank=hvd.rank(),
            seed_fn=seed_fn)

        val_ds = KerasSequenceLoader(
            f'{args.data_dir}/val',
            batch_size=args.batch_size,
            label_names=LABEL_COLUMNS,
            cat_names=CATEGORICAL_COLUMNS,
            cont_names=CONTINUOUS_COLUMNS,
            engine="parquet",
            shuffle=False,
            buffer_size=0.06,  # how many batches to load at once
            parts_per_chunk=1,
            global_size=hvd.size(),
            global_rank=hvd.rank())

        history = model.fit(train_ds,
                            validation_data=val_ds,
                            steps_per_epoch=int(train_rows / args.batch_size / hvd.size()),
                            validation_steps=int(val_rows / args.batch_size / hvd.size()),
                            callbacks=callbacks,
                            verbose=verbose,
                            epochs=args.epochs)

    if hvd.rank() == 0:
        return history.history

### Wrapper function to train

Here we call the `horovod.spark.run` to start the training process in Horovod on Spark.

In [6]:
def train(dimensions, train_rows, val_rows, args):
    # Horovod: run training.
    history = horovod.spark.run(train_fn,
                                args=(dimensions, train_rows, val_rows, args),
                                num_proc=args.num_proc,
                                extra_mpi_args='-mca btl_tcp_if_include enp134s0f0 -x NCCL_IB_GID_INDEX=3',
                                stdout=sys.stdout,
                                stderr=sys.stderr,
                                verbose=2,
                                nics={},
                                prefix_output_with_timestamp=True)[0]

    best_val_loss = min(history['val_loss'])
    print('Best Loss: %f' % best_val_loss)

## Use NVTabular

Here we set `--dataloader` to `nvtabular` to force NVTabular run.

In [7]:
def main():
    parser = argparse.ArgumentParser(description='Criteo Spark Keras Training Example',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--data-dir', default='file:///opt/data/criteo/parquet',
                        help='location of the transformed Criteo dataset in Parquet format')
    parser.add_argument('--logs-dir', default='/opt/experiments/criteo', help='location of TensorFlow logs')
    parser.add_argument('--dataloader', default=PETASTORM_DATALOADER,
                        choices=[PETASTORM_DATALOADER, NVTABULAR_DATALOADER],
                        help='dataloader to use')
    parser.add_argument('--num-proc', type=int, default=1, help='number of worker processes for training')
    parser.add_argument('--learning-rate', type=float, default=0.0001, help='initial learning rate')
    parser.add_argument('--batch-size', type=int, default=64 * 1024, help='batch size')
    parser.add_argument('--epochs', type=int, default=3, help='number of epochs to train')
    parser.add_argument('--local-checkpoint-file', default='checkpoint', help='model checkpoint')
    args = parser.parse_args(args=['--num-proc', '16', '--data-dir', 'file:///raid/spark-team/criteo/parquet', 
                                   '--dataloader', 'nvtabular', '--learning-rate', '0.001',
                                   '--batch-size', '65535','--epochs', '1', '--logs-dir', 'tf_logs',
                                   '--local-checkpoint-file', 'ckpt_file'])
                                   

    dimensions = get_category_dimensions(spark, args.data_dir)

    train_df = spark.read.parquet(f'{args.data_dir}/train')
    val_df = spark.read.parquet(f'{args.data_dir}/val')
    test_df = spark.read.parquet(f'{args.data_dir}/test')
    train_rows, val_rows, test_rows = train_df.count(), val_df.count(), test_df.count()
    print('Training: %d' % train_rows)
    print('Validation: %d' % val_rows)
    print('Test: %d' % test_rows)

    train(dimensions, train_rows, val_rows, args)

    spark.stop()

In [8]:
main()

21/09/06 09:03:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


{'c0': '7912888',
 'c1': '33822',
 'c10': '582468',
 'c11': '245827',
 'c12': '10',
 'c13': '2208',
 'c14': '10666',
 'c15': '103',
 'c16': '3',
 'c17': '967',
 'c18': '14',
 'c19': '8165895',
 'c2': '17138',
 'c20': '2675939',
 'c21': '7156452',
 'c22': '302515',
 'c23': '12021',
 'c24': '96',
 'c25': '34',
 'c3': '7338',
 'c4': '20045',
 'c5': '3',
 'c6': '7104',
 'c7': '1381',
 'c8': '62',
 'c9': '5554113'}


Training: 4195197692
Validation: 89137318
Test: 89137319


Checking whether extension tensorflow was built with MPI.
Extension tensorflow was built with MPI.
mpirun --allow-run-as-root --tag-output -np 16 -H dgx2h0194-a1adff968d508e8d1142986f3e2c42dc:16 -bind-to none -map-by slot -mca pml ob1 -mca btl ^openib --timestamp-output      -mca btl_tcp_if_include enp134s0f0 -x NCCL_IB_GID_INDEX=3 -x NCCL_DEBUG=INFO -mca plm_rsh_agent "/home/ngc-auth-ldap-allxu/miniconda3/bin/python -m horovod.spark.driver.mpirun_rsh gAWVcAEAAAAAAAB9lCiMAmxvlF2UjAkxMjcuMC4wLjGUTWIShpRhjAdlbnA1M3MwlF2UjAwxMC4xNDguMzAuNTmUTWIShpRhjAdlbnA1OHMwlF2UjAwxMC4xNDguOTQuNTmUTWIShpRhjAdlbnA4OHMwlF2UjAwxMC4xNDkuMzAuMzSUTWIShpRhjAdlbnA5M3MwlF2UjAwxMC4xNDkuOTQuNTeUTWIShpRhjAplbnAxMzRzMGYwlF2UjAsxMC4xNTAuMzAuMpRNYhKGlGGMCGVucDE4NHMwlF2UjA0xMC4xNDguMTU4LjU5lE1iEoaUYYwIZW5wMTg5czCUXZSMDTEwLjE0OC4yMjIuNTmUTWIShpRhjAhlbnAyMjVzMJRdlIwNMTAuMTQ5LjE1OC41N5RNYhKGlGGMCGVucDIzMHMwlF2UjA0xMC4xNDkuMjIyLjU3lE1iEoaUYYwHZG9ja2VyMJRdlIwKMTcyLjE3LjAuMZRNYhKGlGF1Lg== gAWVAwMAAAAAAACMI2hvcm92b2QucnVubmV

Mon Sep  6 09:03:56 2021[1,9]<stdout>:Changing cwd from /home/ngc-auth-ldap-allxu to /raid/spark-team/allen-dlrm/spark-3.1.2-bin-hadoop3.2/work/app-20210906090316-0000/1
Mon Sep  6 09:03:56 2021[1,12]<stdout>:Changing cwd from /home/ngc-auth-ldap-allxu to /raid/spark-team/allen-dlrm/spark-3.1.2-bin-hadoop3.2/work/app-20210906090316-0000/1


...

Mon Sep  6 09:05:10 2021[1,11]<stderr>:2021-09-06 09:05:10.585794: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tf_logs/plugins/profile/2021_09_06_09_05_10
Mon Sep  6 09:05:10 2021[1,11]<stderr>:
Mon Sep  6 09:05:10 2021[1,15]<stderr>:2021-09-06 09:05:10.586581: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tf_logs/plugins/profile/2021_09_06_09_05_10
Mon Sep  6 09:05:10 2021[1,15]<stderr>:
Mon Sep  6 09:05:10 2021[1,3]<stderr>:2021-09-06 09:05:10.587215: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to tf_logs/plugins/profile/2021_09_06_09_05_10/dgx2h0194.memory_profile.json.gz
Mon Sep  6 09:05:10 2021[1,12]<stderr>:2021-09-06 09:05:10.589198: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: tf_logs/plugins/profile/2021_09_06_09_05_10
Mon Sep  6 09:05:10 2021[1,12]<stderr>:Dumped tool data for xplane.pb to tf_logs/plugins/p

Mon Sep  6 09:05:10 2021[1,8]<stderr>:
Mon Sep  6 09:05:10 2021[1,8]<stderr>:2021-09-06 09:05:10.609086: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to tf_logs/plugins/profile/2021_09_06_09_05_10/dgx2h0194.memory_profile.json.gz
Mon Sep  6 09:05:10 2021[1,10]<stderr>:2021-09-06 09:05:10.609124: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to tf_logs/plugins/profile/2021_09_06_09_05_10/dgx2h0194.memory_profile.json.gz
Mon Sep  6 09:05:10 2021[1,1]<stderr>:2021-09-06 09:05:10.609444: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to tf_logs/plugins/profile/2021_09_06_09_05_10/dgx2h0194.memory_profile.json.gz
Mon Sep  6 09:05:10 2021[1,8]<stderr>:2021-09-06 09:05:10.614234: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: tf_logs/plugins/profile/2021_09_06_09_05_

Mon Sep  6 09:05:11 2021[1,12]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.1455s vs `on_train_batch_begin` time: 0.1987s). Check your callbacks.
Mon Sep  6 09:05:11 2021[1,12]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1455s vs `on_train_batch_end` time: 1.6635s). Check your callbacks.
Mon Sep  6 09:05:11 2021[1,3]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.1708s vs `on_train_batch_begin` time: 0.2090s). Check your callbacks.
Mon Sep  6 09:05:11 2021[1,3]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1708s vs `on_train_batch_end` time: 1.6582s). Check your callbacks.
Mon Sep  6 09:05:11 2021[1,10]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time:

4000/4000 [==============================]Mon Sep  6 09:18:36 2021[1,0]<stdout>: - 845s 204ms/step - loss: 0.1270 - auc: 0.7832 - val_loss: 0.1273 - val_auc: 0.7933  6 09:18:22 2021[1,0]<stdout>


Best Loss: 0.127242


## Use Petastorm

Here we set `--dataloader` to `petastorm` to force Petastorm run.

In [9]:
def main_petastorm():
    parser = argparse.ArgumentParser(description='Criteo Spark Keras Training Example',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--data-dir', default='file:///opt/data/criteo/parquet',
                        help='location of the transformed Criteo dataset in Parquet format')
    parser.add_argument('--logs-dir', default='/opt/experiments/criteo', help='location of TensorFlow logs')
    parser.add_argument('--dataloader', default=PETASTORM_DATALOADER,
                        choices=[PETASTORM_DATALOADER, NVTABULAR_DATALOADER],
                        help='dataloader to use')
    parser.add_argument('--num-proc', type=int, default=1, help='number of worker processes for training')
    parser.add_argument('--learning-rate', type=float, default=0.0001, help='initial learning rate')
    parser.add_argument('--batch-size', type=int, default=64 * 1024, help='batch size')
    parser.add_argument('--epochs', type=int, default=3, help='number of epochs to train')
    parser.add_argument('--local-checkpoint-file', default='checkpoint', help='model checkpoint')
    args = parser.parse_args(args=['--num-proc', '16', '--data-dir', 'file:///raid/spark-team/criteo/parquet', 
                                   '--dataloader', 'petastorm', '--learning-rate', '0.001',
                                   '--batch-size', '65535','--epochs', '1', '--logs-dir', 'tf_logs',
                                   '--local-checkpoint-file', 'ckpt_file'])
                                   

    dimensions = get_category_dimensions(spark, args.data_dir)

    train_df = spark.read.parquet(f'{args.data_dir}/train')
    val_df = spark.read.parquet(f'{args.data_dir}/val')
    test_df = spark.read.parquet(f'{args.data_dir}/test')
    train_rows, val_rows, test_rows = train_df.count(), val_df.count(), test_df.count()
    print('Training: %d' % train_rows)
    print('Validation: %d' % val_rows)
    print('Test: %d' % test_rows)

    train(dimensions, train_rows, val_rows, args)

    spark.stop()

In [10]:
main_petastorm()

21/09/06 11:44:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


{'c0': '7912888',
 'c1': '33822',
 'c10': '582468',
 'c11': '245827',
 'c12': '10',
 'c13': '2208',
 'c14': '10666',
 'c15': '103',
 'c16': '3',
 'c17': '967',
 'c18': '14',
 'c19': '8165895',
 'c2': '17138',
 'c20': '2675939',
 'c21': '7156452',
 'c22': '302515',
 'c23': '12021',
 'c24': '96',
 'c25': '34',
 'c3': '7338',
 'c4': '20045',
 'c5': '3',
 'c6': '7104',
 'c7': '1381',
 'c8': '62',
 'c9': '5554113'}


Training: 4195197692
Validation: 89137318
Test: 89137319


Checking whether extension tensorflow was built with MPI.
Extension tensorflow was built with MPI.
mpirun --allow-run-as-root --tag-output -np 16 -H dgx2h0194-a1adff968d508e8d1142986f3e2c42dc:16 -bind-to none -map-by slot -mca pml ob1 -mca btl ^openib --timestamp-output      -mca btl_tcp_if_include enp134s0f0 -x NCCL_IB_GID_INDEX=3 -x NCCL_DEBUG=INFO -mca plm_rsh_agent "/home/ngc-auth-ldap-allxu/miniconda3/bin/python -m horovod.spark.driver.mpirun_rsh gAWVcAEAAAAAAAB9lCiMAmxvlF2UjAkxMjcuMC4wLjGUTU5+hpRhjAdlbnA1M3MwlF2UjAwxMC4xNDguMzAuNTmUTU5+hpRhjAdlbnA1OHMwlF2UjAwxMC4xNDguOTQuNTmUTU5+hpRhjAdlbnA4OHMwlF2UjAwxMC4xNDkuMzAuMzSUTU5+hpRhjAdlbnA5M3MwlF2UjAwxMC4xNDkuOTQuNTeUTU5+hpRhjAplbnAxMzRzMGYwlF2UjAsxMC4xNTAuMzAuMpRNTn6GlGGMCGVucDE4NHMwlF2UjA0xMC4xNDguMTU4LjU5lE1OfoaUYYwIZW5wMTg5czCUXZSMDTEwLjE0OC4yMjIuNTmUTU5+hpRhjAhlbnAyMjVzMJRdlIwNMTAuMTQ5LjE1OC41N5RNTn6GlGGMCGVucDIzMHMwlF2UjA0xMC4xNDkuMjIyLjU3lE1OfoaUYYwHZG9ja2VyMJRdlIwKMTcyLjE3LjAuMZRNTn6GlGF1Lg== gAWVAwMAAAAAAACMI2hvcm92b2QucnVubmV

Mon Sep  6 11:44:58 2021[1,12]<stdout>:Changing cwd from /home/ngc-auth-ldap-allxu to /raid/spark-team/allen-dlrm/spark-3.1.2-bin-hadoop3.2/work/app-20210906114426-0004/1
Mon Sep  6 11:44:58 2021[1,13]<stdout>:Changing cwd from /home/ngc-auth-ldap-allxu to /raid/spark-team/allen-dlrm/spark-3.1.2-bin-hadoop3.2/work/app-20210906114426-0004/1


...

Mon Sep  6 11:46:14 2021[1,4]<stderr>:2021-09-06 11:46:14.031628: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to tf_logs/plugins/profile/2021_09_06_11_46_12/dgx2h0194.trace.json.gz
Mon Sep  6 11:46:14 2021[1,13]<stderr>:2021-09-06 11:46:14.035052: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to tf_logs/plugins/profile/2021_09_06_11_46_12/dgx2h0194.memory_profile.json.gz
Mon Sep  6 11:46:14 2021[1,13]<stderr>:2021-09-06 11:46:14.044101: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: tf_logs/plugins/profile/2021_09_06_11_46_12
Mon Sep  6 11:46:14 2021[1,13]<stderr>:Dumped tool data for xplane.pb to tf_logs/plugins/profile/2021_09_06_11_46_12/dgx2h0194.xplane.pb
Mon Sep  6 11:46:14 2021[1,13]<stderr>:Dumped tool data for overview_page.pb to tf_logs/plugins/profile/2021_09_06_11_46_12/dgx2h0194.overview_page.pb
Mon Sep  6 11:46:14 20

Mon Sep  6 11:46:14 2021[1,15]<stderr>:
Mon Sep  6 11:46:14 2021[1,11]<stderr>:2021-09-06 11:46:14.192490: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: tf_logs/plugins/profile/2021_09_06_11_46_12
Mon Sep  6 11:46:14 2021[1,11]<stderr>:Dumped tool data for xplane.pb to tf_logs/plugins/profile/2021_09_06_11_46_12/dgx2h0194.xplane.pb
Mon Sep  6 11:46:14 2021[1,11]<stderr>:Dumped tool data for overview_page.pb to tf_logs/plugins/profile/2021_09_06_11_46_12/dgx2h0194.overview_page.pb
Mon Sep  6 11:46:14 2021[1,11]<stderr>:Dumped tool data for input_pipeline.pb to tf_logs/plugins/profile/2021_09_06_11_46_12/dgx2h0194.input_pipeline.pb
Mon Sep  6 11:46:14 2021[1,11]<stderr>:Dumped tool data for tensorflow_stats.pb to tf_logs/plugins/profile/2021_09_06_11_46_12/dgx2h0194.tensorflow_stats.pb
Mon Sep  6 11:46:14 2021[1,11]<stderr>:Dumped tool data for kernel_stats.pb to tf_logs/plugins/profile/2021_09_06_11_46_12/dgx2h0194.kernel_stats.pb
Mon Sep  6 11:46:14 

Mon Sep  6 11:46:21 2021[1,5]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 1.5142s vs `on_train_batch_end` time: 2.2162s). Check your callbacks.
Mon Sep  6 11:46:21 2021[1,15]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 1.4938s vs `on_train_batch_end` time: 2.2278s). Check your callbacks.
Mon Sep  6 11:46:21 2021[1,9]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 1.5230s vs `on_train_batch_end` time: 2.1673s). Check your callbacks.
Mon Sep  6 11:46:21 2021[1,12]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 1.5441s vs `on_train_batch_end` time: 2.1769s). Check your callbacks.
Mon Sep  6 11:46:21 2021[1,8]<stderr>:WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 1.5099s 

Mon Sep  6 21:09:31 2021[1,0]<stdout>:4000/4000 [==============================]Mon Sep  6 21:09:31 2021[1,0]<stdout>: - 33843s 8s/step - loss: 0.1291 - auc: 0.7793 - val_loss: 0.1287 - val_auc: 0.7880  6 21:09:31 2021[1,0]<stdout>:


Best Loss: 0.128178


### Conclusion

From cell[8] and cell[10], we can see the total training time respectively: 

- NVTabular: 845s
- Petastorm: 33843s 

So the speedup : 33843 / 845 = `40.05`